## Классификация текстов с использованием предобученных языковых моделей.

В данном задании вам предстоит обратиться к задаче классификации текстов и решить ее с использованием предобученной модели BERT.

In [ ]:
import json
# do not change the code in the block below
# __________start of block__________
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

%matplotlib inline
# __________end of block__________

Обратимся к набору данных SST-2. Holdout часть данных (которая понадобится вам для посылки) доступна по ссылке ниже.

In [ ]:
# do not change the code in the block below
# __________start of block__________

!wget https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
# __________end of block__________

In [3]:
# do not change the code in the block below
# __________start of block__________
df = pd.read_csv(
    "https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv",
    delimiter="\t",
    header=None,
)
texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]
texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]
with open("texts_holdout.json") as iofile:
    texts_holdout = json.load(iofile)
# __________end of block__________

Весь остальной код предстоит написать вам.

Для успешной сдачи на максимальный балл необходимо добиться хотя бы __84.5% accuracy на тестовой части выборки__.

In [4]:
!pip install --upgrade keras_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 7.4 MB/s eta 0:00:00   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 68.4 MB/s eta 0:00:00   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.0 MB/s eta 0:00:00   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.2 MB/s eta 0:00:00ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.

In [5]:
!pip install --upgrade tensorflow keras_nlp

In [6]:
import tensorflow as tf
import keras
import keras_nlp

In [7]:
from keras_hub.models import BertClassifier, BertTextClassifierPreprocessor, BertPreprocessor

In [8]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_base_en_uncased",
    sequence_length=32
)

100%|██████████| 510/510 [00:00<00:00, 154kB/s]

100%|██████████| 547/547 [00:00<00:00, 1.10MB/s]

100%|██████████| 226k/226k [00:00<00:00, 1.32MB/s]

In [9]:
#x_train = preprocessor(texts_train)
#x_test = preprocessor(texts_test)

In [10]:
#x_train

In [11]:
#y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
#y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)

In [12]:
num_classes = 2
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_en_uncased",
    num_classes=num_classes,
    activation="softmax",
    preprocessor=preprocessor
)

100%|██████████| 418M/418M [00:07<00:00, 55.7MB/s]

In [13]:
#classifier = BertTextClassifier.from_preset(
    #"bert_base_en",
    #num_classes=2,
    #preprocessor=preprocessor,
    #activation='softmax'
#)

In [14]:
classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-5),  # Low learning rate for fine-tuning
    loss="sparse_categorical_crossentropy",  # Suitable for integer labels
    metrics=["accuracy"]
)

In [15]:
history = classifier.fit(
    x=texts_train,
    y=y_train,
    batch_size=4,
    epochs=2,
    verbose=1
)

Epoch 1/2  41/1250 ━━━━━━━━━━━━━━━━━━━━ 1:21:04 4s/step - accuracy: 0.5164 - loss: 0.7037

KeyboardInterrupt: 

#### Сдача взадания в контест
Сохраните в словарь `out_dict` вероятности принадлежности к первому (положительному) классу

In [ ]:
out_dict = {
    'train': # list of length 5000 with probas
    'test': # list of length 1920 with probas
    'holdout': # list of length 500 with probas
}

Несколько `assert`'ов для проверки вашей посылки:

In [ ]:
assert isinstance(out_dict["train"], list), "Object must be a list of floats"
assert isinstance(out_dict["train"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["train"]) == 5000
), "The predicted probas list length does not match the train set size"

assert isinstance(out_dict["test"], list), "Object must be a list of floats"
assert isinstance(out_dict["test"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["test"]) == 1920
), "The predicted probas list length does not match the test set size"

assert isinstance(out_dict["holdout"], list), "Object must be a list of floats"
assert isinstance(out_dict["holdout"][0], float), "Object must be a list of floats"
assert len(
    out_dict["holdout"] == 500
), "The predicted probas list length does not match the holdout set size"

Запустите код ниже для генерации посылки.

In [ ]:
# do not change the code in the block below
# __________start of block__________
FILENAME = "submission_dict_hw_text_classification_with_bert.json"

with open(FILENAME, "w") as iofile:
    json.dump(out_dict, iofile)
print(f"File saved to `{FILENAME}`")
# __________end of block__________

На этом задание завершено. Поздравляем!